# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "../Resources/schools_complete.csv"
student_data_to_load = "../Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [ ]:
school_data.head()

In [ ]:
school_data_complete.head()

In [ ]:
# Calculate the total number of schools
total_schools = school_data["School ID"].count()

total_schools

In [ ]:
# Calculate the total budget
school_budget = school_data["budget"].sum()

school_budget

In [ ]:
# Calculate the total number of students
total_students = school_data_complete["student_name"].count()

total_students

In [ ]:
# Calculate the average math score
average_math = school_data_complete["math_score"].mean()

round(average_math)

In [ ]:
# Calculate the average reading score
average_reading = school_data_complete["reading_score"].mean()

round(average_reading)

In [ ]:
# Calculate the overall passing rate (overall average score)

overall_passing_rate = (average_reading + average_math) / 2

clean_rate = round((overall_passing_rate),2)

clean_rate

In [ ]:
# Calculate the percentage of students with a passing math score (70 or greater)

passing_math = (school_data_complete["math_score"] >= 70)
passing_reading = (school_data_complete["reading_score"] >= 70)

per_passing_math = round((passing_math.sum()/total_students)*100, 2)

per_passing_reading = round((passing_reading.sum()/total_students)*100, 2)





In [ ]:
summary_df = pd.DataFrame(
    {"Total Schools": [total_schools],
     "Total Students": [total_students],
     "Total Budget": [school_budget],
     "Average Math Score": [average_math],
     "Average Reading Score": [average_reading],
     "Overall Average Score": [clean_rate],
     "Precentage Passing Math (Scores Over 70)": [per_passing_math],
     "Percentage Passing Reading (Scores Over 70)": [per_passing_reading],
     }
)
summary_df

In [ ]:
# Figure out mapping issues
summary_df["Total Students"] = summary_df["Total Students"].map("${:,.2f}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [ ]:
merged_school_data = pd.merge(school_data, student_data, how='outer', on="school_name")
merged_school_data.head()

In [ ]:
# total students per school, finding the percent over 70 

# students_over_70 = merged_school_data["math_score"] >= 70

# students_over_70

#  merged_school_data.set_index('school_name')

# school_groups = merged_school_data(['school_name']).sum()/[students_over_70]

In [ ]:
# total students per school, finding the percent over 70 

students_over_70_math = (merged_school_data["math_score"] >= 70) * 100
students_over_70_reading = (merged_school_data["reading_score"] >= 70) * 100

In [ ]:
merged_school_data["Pass Math"] = students_over_70_math
merged_school_data["Pass Reading"] = students_over_70_reading

merged_school_data

In [ ]:
average_scores_schools = merged_school_data.groupby(['school_name'])['Pass Math','Pass Reading']

average_scores_schools.mean()

In [ ]:
total_students_per_school = merged_school_data["school_name"].value_counts()

total_students_per_school

In [ ]:
total_budgets = merged_school_data.groupby(['school_name'])['budget'].mean()

total_budgets

In [ ]:
per_student_budget = [total_budgets/total_students_per_school]
per_student_budget

In [ ]:
average_school_math_reading = merged_school_data.groupby(['school_name'])['size','math_score', 'reading_score', 'Pass Math', "Pass Reading", 'budget'].mean()
round(average_school_math_reading, 2)

In [ ]:
# Average passing rate
average_school_math_reading["% Overall Passing rate"] = (average_school_math_reading["Pass Math"] +
average_school_math_reading["Pass Reading"]) / 2

average_school_math_reading["Budget Per Student"] = average_school_math_reading["budget"] / average_school_math_reading['size']
 
average_school_math_reading

In [ ]:
list(average_school_math_reading.index.values)

In [ ]:
schools_df = pd.merge(average_school_math_reading, school_data, how = 'outer', on=['school_name'])

schools_df

In [ ]:
schools_df = schools_df.rename(columns={"school_name": "School Name", "math_score": "Math Score",
                                        "reading_score":"Reading Score", "type":"Type", "size_y":"Size", "budget_y":"Budget"})
schools_df

In [ ]:
# Deleting extra/duplicate columns

del schools_df['size_x']
del schools_df['budget_x']

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [ ]:
top_five_schools = schools_df.sort_values(by=['% Overall Passing rate'], ascending=False).head(5)

top_five_schools

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [ ]:
bottom_five_schools = schools_df.sort_values(by=['% Overall Passing rate'], ascending=True).head(5)
bottom_five_schools

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
math_scores_by_grade_df = pd.pivot_table(student_data, values=["math_score"], index=["school_name"],
                                        columns=["grade"])

math_scores_by_grade_df

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
reading_scores_by_grade_df = pd.pivot_table(student_data, values=["reading_score"], index=["school_name"],
                                        columns=["grade"])

reading_scores_by_grade_df

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
bins = [0, 585, 615, 645, 675]
spending_labels = ["$0-585", "$585-615", "$615-645", "$645-675"]

In [ ]:
bins_school_spending = pd.cut(schools_df['Budget Per Student'], bins, labels=spending_labels)

bins_school_spending = pd.DataFrame(bins_school_spending)

schools_df['Spending Level'] = bins_school_spending

bins_school_spending

In [ ]:
scores_by_school_spending = schools_df.groupby(['Spending Level'])['Reading Score',
                                                        'Math Score',
                                                        'Pass Reading',
                                                        'Pass Math',
                                                        '% Overall Passing rate'
                                                        ].mean()
scores_by_school_spending

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
bins_school_size = pd.cut(schools_df['Size'], size_bins, labels=group_names)

bins_school_size = pd.DataFrame(bins_school_size)

schools_df['Size Level'] = bins_school_size

bins_school_size

In [ ]:
scores_by_school_size = schools_df.groupby(['Size Level'])['Reading Score',
                                                        'Math Score',
                                                        'Pass Reading',
                                                        'Pass Math',
                                                        '% Overall Passing rate'
                                                        ].mean()
scores_by_school_size

## Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
scores_by_school_size = schools_df.groupby(['Type'])['Reading Score',
                                                        'Math Score',
                                                        'Pass Reading',
                                                        'Pass Math',
                                                        '% Overall Passing rate'
                                                        ].mean()
scores_by_school_size

In [ ]:
# Observation 1:

# Students in charter schools are much more likely to pass their reading and math classes than their disctrict counter parts.

# Observation 2:

# Schools over 200 students have the lowest overall passing rate.

# Weird Observation:

# According to this data, schools that are in the highest per student spending bracket produce the lowest average passing rate.